In [ ]:
# !pip install seaborn
# !pip install sklearn
# !pip install --user scikit-learn
# !pip install matplotlib
# !pip install nltk

In [ ]:
import pandas as pd
import numpy as np
import string
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from collections import Counter
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [ ]:
data = pd.read_csv('../input/sms-spam-collection-dataset/spam.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
data.columns = ["class","message"]

In [ ]:
data.head()

In [ ]:
data['length'] = [len(d) for d in data.message]

In [ ]:
data.head()

In [ ]:
data.groupby('class').describe()

In [ ]:
sns.countplot(x=data['class'])

In [ ]:
sns.barplot(x=data["class"],y=data["length"])

In [ ]:
data.drop(["length"],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
def transform_message(message):
    message_not_punc = [] # Message without punctuation
    i = 0
    for punctuation in message:
        if punctuation not in string.punctuation:
            message_not_punc.append(punctuation)
    # Join words again to form the string.
    message_not_punc = ''.join(message_not_punc) 

    # Remove any stopwords for message_not_punc, but first we should     
    # to transform this into the list.
    message_clean = list(message_not_punc.split(" "))
    while i <= len(message_clean):
        for mess in message_clean:
            if mess.lower()  in stopwords.words('english'):
                message_clean.remove(mess)
        i =i +1
    return  message_clean

In [ ]:
data['message'].apply(transform_message)

In [ ]:
vectorization = CountVectorizer(analyzer=transform_message)

In [ ]:
X = vectorization.fit(data["message"])

In [ ]:
X_transform = X.transform(data["message"])

In [ ]:
print(X_transform)

In [ ]:
tfidf_transformer = TfidfTransformer().fit(X_transform)

In [ ]:
X_tfidf = tfidf_transformer.transform(X_transform)

In [ ]:
print(X_tfidf.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['message'], data['class'], test_size=0.3,random_state = 50, stratify=data['class'])  

In [ ]:
X_train,y_train

In [ ]:
bow = CountVectorizer(stop_words='english')

In [ ]:
# Fit the bag of words on the training docs
bow.fit(X_train)

In [ ]:
X_train = bow.transform(X_train)
X_test = bow.transform(X_test)

In [ ]:
X_train,X_test

****Naive Bayes Classifier

In [ ]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)

In [ ]:
print(f'Accuracy : {accuracy_score(y_test, naive_bayes.predict(X_test)):.3f}')
print(f'Precision : {precision_score(y_test, naive_bayes.predict(X_test), pos_label="spam"):.3f}')
print(f'Recall : {recall_score(y_test, naive_bayes.predict(X_test), pos_label="spam"):.3f}')
print(f'F1-Score : {f1_score(y_test, naive_bayes.predict(X_test), pos_label="spam"):.3f}')

In [ ]:
predictions = naive_bayes.predict(X_test)

In [ ]:
print(predictions)

In [ ]:
print (classification_report(y_test, predictions))

****Support Vector Machine Classifier

In [ ]:
svm_clf = SVC(kernel='linear').fit(X_train, y_train)

In [ ]:
print(f'Accuracy : {accuracy_score(y_test, svm_clf.predict(X_test)):.3f}')
print(f'Precision : {precision_score(y_test, svm_clf.predict(X_test), pos_label="spam"):.3f}')
print(f'Recall : {recall_score(y_test, svm_clf.predict(X_test), pos_label="spam"):.3f}')
print(f'F1-Score : {f1_score(y_test, svm_clf.predict(X_test), pos_label="spam"):.3f}')

In [ ]:
svm_predictions = svm_clf.predict(X_test)

In [ ]:
print (classification_report(y_test, svm_predictions))